In [1]:
!pip install groq rouge-score bert-score nltk
import nltk
nltk.download('punkt')

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.3/137.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.1 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=850eb4dd7ef813b62210ff468b2fcb832429e13d6301fb288932a157cdd03fc4
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
%%javascript
function ClickConnect(){
  console.log("Clicking");
  document.querySelector("colab-toolbar-button#connect").click();
}
setInterval(ClickConnect, 60000)

<IPython.core.display.Javascript object>

In [3]:
#####################################################################
# 1. MOUNT DRIVE & IMPORTS
#####################################################################
from google.colab import drive
drive.mount("/content/drive")

import os, re, json, time, logging
from datetime import datetime
from pathlib import Path
from typing import List, Dict, Any

import pandas as pd
import numpy as np
from groq import Groq   # Groq client


#####################################################################
# 2. PATHS & API KEY
#####################################################################
INPUT_FILE = "/content/drive/MyDrive/Final Thesis Code/Input/clean_input_30.xlsx"

BASE_OUT = Path("/content/drive/MyDrive/Final Thesis Code/Output/Zero Shot Prompting/llama-4-scout-17b-16e-instruct/")
BASE_OUT.mkdir(parents=True, exist_ok=True)

FINAL_OUTPUT_FILE = BASE_OUT / "llama-4-scout-17b-16e-instruct_zero_shot_full_output.xlsx"

API_KEY_PATH = "/content/drive/MyDrive/Final Thesis Code/api_keys/groq_key1.txt"

def load_key(path: str):
    with open(path, "r") as f:
        return f.read().strip()

API_KEY = load_key(API_KEY_PATH)
client = Groq(api_key=API_KEY)

print("Input:", INPUT_FILE)
print("Output folder:", BASE_OUT)
print("Groq API key loaded ✓")


#####################################################################
# 3. GLOBAL CONFIG
#####################################################################
MODEL_NAME     = "meta-llama/llama-4-maverick-17b-128e-instruct"
MAX_CHARS      = 2300
GLOBAL_MIN_GAP = 45   # Groq is faster, but we keep a safety gap
LAST_TS        = 0.0

VALID_TOPICS = [
    "Natural Language Processing","Artificial Intelligence","Prompt Engineering",
    "Machine Learning","Deep Learning","Reinforcement Learning","Generative AI",
    "Data Science","Time Series","Statistics","LangChain","Langraph",
    "Python Programming","Mlops","Agentic AI","Other"
]


#####################################################################
# 4. LOGGING
#####################################################################
def setup_logging():
    logs = Path("/content/logs")
    logs.mkdir(exist_ok=True)
    logfile = logs / f"log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"

    logging.basicConfig(
        level=logging.INFO,
        handlers=[logging.FileHandler(logfile, encoding="utf-8"),
                  logging.StreamHandler()],
        format="%(asctime)s | %(levelname)s | %(message)s"
    )
    return logging.getLogger(__name__)

logger = setup_logging()


#####################################################################
# 5. CLEANING & CHUNKING
#####################################################################
def deep_clean(text: str) -> str:
    t = str(text)
    t = re.sub(r"https?://\S+|www\.\S+", " ", t)
    t = re.sub(r"\b[\w\.-]+@[\w\.-]+\.\w+\b", " ", t)
    t = re.sub(r"\[.*?\]|\(.*?\)", " ", t)
    t = re.sub(r"\s+", " ", t)

    t = re.sub(r"\bNLP\b", "Natural Language Processing (NLP)", t)
    t = re.sub(r"\bML\b", "Machine Learning (ML)", t)
    t = re.sub(r"\bAI\b", "Artificial Intelligence (AI)", t)
    return t.strip()


def chunk_text(text: str, max_chars=MAX_CHARS) -> List[str]:
    clean = deep_clean(text)
    if len(clean) <= max_chars:
        return [clean] if clean else [""]

    sents = [s.strip() for s in re.split(r"(?<=[.!?])\s+", clean) if s.strip()]
    chunks, cur = [], ""

    for s in sents:
        if len(cur) + len(s) + 1 <= max_chars:
            cur = (cur + " " + s).strip()
        else:
            if cur:
                chunks.append(cur)
            cur = s

    if cur:
        chunks.append(cur)
    return chunks or [""]


#####################################################################
# 6. JSON EXTRACTION
#####################################################################
def extract_json(text: str) -> Dict[str, Any]:
    if not text:
        return {}
    start = text.find("{")
    end = text.rfind("}")
    if start == -1 or end == -1 or end <= start:
        return {}
    try:
        return json.loads(text[start:end+1])
    except:
        return {}


#####################################################################
# 7. GROQ CALL (GLOBAL WAIT + RETRIES)
#####################################################################
def groq_call(prompt: str, temperature=0.15, retries=3) -> str:
    global LAST_TS
    now = time.time()

    if LAST_TS > 0 and now - LAST_TS < GLOBAL_MIN_GAP:
        wait = GLOBAL_MIN_GAP - (now - LAST_TS)
        logger.info(f"Waiting {wait:.1f}s before next request")
        time.sleep(wait)

    for attempt in range(1, retries+1):
        try:
            resp = client.chat.completions.create(
                model=MODEL_NAME,
                messages=[{"role": "user", "content": prompt}],
                temperature=temperature,
                max_tokens=2048
            )
            LAST_TS = time.time()
            return resp.choices[0].message.content.strip()

        except Exception as e:
            logger.warning(f"Groq call failed ({attempt}/{retries}): {e}")
            time.sleep(4 * attempt)

    logger.error("Groq failed after retries — returning empty string.")
    return ""


#####################################################################
# 8. ZERO-SHOT PROMPT (ALL TASKS IN ONE CALL)
#####################################################################
def generate_zero_shot(transcript: str):
    topics_short = ", ".join(VALID_TOPICS)

    prompt = (
        "You are an expert NLP assistant.\n"
        "Perform ALL tasks concisely using ONLY the transcript.\n\n"
        "Respond in a JSON-like object with EXACTLY these keys:\n"
        "generated_summary, predicted_topics, generated_questions, key_concepts.\n\n"
        "Requirements:\n"
        "- generated_summary: 3–5 sentence abstractive summary.\n"
        f"- predicted_topics: 1–3 most relevant from [{topics_short}]. Use exact labels.\n"
        "- generated_questions: 3 short Q&A pairs with keys q and a.\n"
        "- key_concepts: 6–10 short noun phrases, no duplicates.\n"
        "- No explanations outside the JSON.\n\n"
        f"Transcript:\n\"\"\"{transcript}\"\"\""
    )

    out = groq_call(prompt, temperature=0.15)
    j = extract_json(out)

    summary = j.get("generated_summary", "").strip()

    topics = j.get("predicted_topics", [])
    if isinstance(topics, str):
        topics = [topics]
    topics = [t for t in topics if t in VALID_TOPICS] or ["Other"]

    qas = j.get("generated_questions", [])
    qa_lines = []
    if isinstance(qas, list):
        for qa in qas:
            q = qa.get("q", "").strip()
            a = qa.get("a", "").strip()
            if q: qa_lines.append(f"Q: {q}")
            if a: qa_lines.append(f"A: {a}")

    concepts = j.get("key_concepts", [])
    if not isinstance(concepts, list):
        concepts = []
    concepts = ", ".join([c.strip() for c in concepts if c.strip()])

    return summary, topics, "\n".join(qa_lines), concepts


#####################################################################
# 9. MAIN PIPELINE — ZERO SHOT (NO EVALUATION)
#####################################################################
def run_pipeline() -> pd.DataFrame:
    df = pd.read_excel(INPUT_FILE)
    results = []

    if FINAL_OUTPUT_FILE.exists():
        old = pd.read_excel(FINAL_OUTPUT_FILE)
        if "row_index" in old.columns:
            done = set(old["row_index"])
            results = old.to_dict(orient="records")
            print(f"Resuming — {len(done)} rows already processed.")
        else:
            done = set()
    else:
        done = set()

    for idx, row in df.iterrows():
        if idx in done:
            print(f"Skipping row {idx}")
            continue

        title = str(row.get("title", "")).strip()
        transcript = str(row.get("transcript", "")).strip()

        print("\n" + "="*80)
        print(f"PROCESSING ROW {idx}: {title}")
        print("="*80)

        try:
            summary, topics, qa_text, concepts = generate_zero_shot(transcript)
        except Exception as e:
            logger.error(f"Error on row {idx}: {e}")
            summary, topics, qa_text, concepts = "", ["Other"], "", ""

        print("\nSUMMARY:\n", summary)
        print("\nTOPICS:\n", topics)
        print("\nQ&A:\n", qa_text)
        print("\nKEY CONCEPTS:\n", concepts)

        rec = {
            "row_index": idx,
            "title": title,
            "summary": summary,
            "topic_classification": ", ".join(topics),
            "Q_and_A": qa_text,
            "key_concepts": concepts
        }

        results.append(rec)
        pd.DataFrame(results).to_excel(FINAL_OUTPUT_FILE, index=False)
        print(f"Saved row {idx}")

    df_out = pd.DataFrame(results)
    df_out.to_excel(FINAL_OUTPUT_FILE, index=False)
    print("DONE. Final file:", FINAL_OUTPUT_FILE)
    return df_out


#####################################################################
# 10. RUN
#####################################################################
df_out = run_pipeline()
print("\nZero-shot Groq pipeline completed ✓")


Mounted at /content/drive
Input: /content/drive/MyDrive/Final Thesis Code/Input/clean_input_30.xlsx
Output folder: /content/drive/MyDrive/Final Thesis Code/Output/Zero Shot Prompting/llama-4-scout-17b-16e-instruct
Groq API key loaded ✓

PROCESSING ROW 0: Reinforcement Learning through Human Feedback - EXPLAINED! | RLHF

SUMMARY:
 The video discusses reinforcement learning with human feedback, using examples and ChatGPT as a practical application. It explains how human feedback accelerates the learning process and guides the algorithm to make informed decisions. The video also delves into the training process of ChatGPT using reinforcement learning and a rewards model.

TOPICS:
 ['Reinforcement Learning', 'Artificial Intelligence', 'Generative AI']

Q&A:
 Q: What is the role of human feedback in reinforcement learning?
A: To guide and accelerate the learning process
Q: What is the primary purpose of the rewards model in ChatGPT?
A: To assess and score the quality of answers generated by

In [4]:
#####################################################################
# 1. IMPORTS
#####################################################################
import os, re, json, warnings
import pandas as pd
import numpy as np

from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from bert_score import score as bert_score

from sklearn.metrics import precision_recall_fscore_support


#####################################################################
# 2. SUPPRESS WARNINGS (BERTScore spam)
#####################################################################
warnings.filterwarnings("ignore")
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)
logging.getLogger("absl").setLevel(logging.ERROR)


#####################################################################
# 3. PATHS (EDIT THESE)
#####################################################################
INPUT_FILE = "/content/drive/MyDrive/Final Thesis Code/Input/clean_input_30.xlsx"
OUTPUT_FILE = "/content/drive/MyDrive/Final Thesis Code/Output/Zero Shot Prompting/llama-4-scout-17b-16e-instruct/llama-4-scout-17b-16e-instruct_zero_shot_full_output.xlsx"
FINAL_EVAL_JSON = "/content/drive/MyDrive/Final Thesis Code/Output/Zero Shot Prompting/llama-4-scout-17b-16e-instruct/evaluation_final.json"

print("Loaded input:", INPUT_FILE)
print("Loaded model output:", OUTPUT_FILE)


#####################################################################
# 4. GOLD TOPIC EXTRACTION (KEYWORD-BASED — FINAL VERSION)
#####################################################################
def gold_topics_from_ref_summary(ref_sum: str):
    text = (ref_sum or "").lower()
    matched = []

    rules = [
        ("Natural Language Processing", [
            "nlp", "bert", "transformer", "language model", "token",
            "text processing", "semantic", "embedding"
        ]),
        ("Artificial Intelligence", [
            "artificial intelligence", "ai system", "symbolic ai",
            "reasoning", "planning", "search"
        ]),
        ("Prompt Engineering", [
            "prompt", "few-shot", "zero-shot", "instruction",
            "cot", "chain-of-thought", "in-context learning"
        ]),
        ("Machine Learning", [
            "machine learning", "supervised", "unsupervised", "regression",
            "classification", "clustering", "features"
        ]),
        ("Deep Learning", [
            "deep learning", "neural network", "cnn", "rnn",
            "lstm", "gan", "transformer model", "backpropagation"
        ]),
        ("Reinforcement Learning", [
            "reinforcement", "policy gradient", "q-learning",
            "reward", "actor-critic", "rlhf"
        ]),
        ("Generative AI", [
            "genai", "text generation", "image generation",
            "diffusion", "sampling", "generation model", "llm"
        ]),
        ("Data Science", [
            "data science", "visualization", "feature", "pandas",
            "analysis", "data preprocessing", "eda"
        ]),
        ("Time Series", [
            "time series", "forecasting", "temporal", "trend",
            "seasonality", "arima", "prophet", "lag"
        ]),
        ("Statistics", [
            "statistics", "probability", "distribution", "variance",
            "hypothesis", "confidence interval", "p-value"
        ]),
        ("LangChain", [
            "langchain", "chain", "memory", "retriever",
            "agent executor", "llmchain", "prompt template"
        ]),
        ("Langraph", [
            "langraph", "workflow", "graph", "multi-agent orchestration",
            "node", "edge", "state graph"
        ]),
        ("Python Programming", [
            "python", "numpy", "matplotlib", "function",
            "loop", "list comprehension", "script"
        ]),
        ("Mlops", [
            "mlops", "deployment", "monitoring", "pipeline",
            "model registry", "cicd", "serving"
        ]),
        ("Agentic AI", [
            "agentic", "tool calling", "multi-agent",
            "planner", "agent", "reasoning agent", "autonomous"
        ])
    ]

    for label, keywords in rules:
        if any(kw in text for kw in keywords):
            matched.append(label)

    return matched or ["Other"]


#####################################################################
# 5. TOKENIZER FOR QA & CONCEPTS
#####################################################################
STOPWORDS = set([
    "the","a","an","in","on","for","to","and","or","of","with","as",
    "by","at","from","that","this","is","are","was","were","be","been",
    "it","its","into","about","over","under","between","across",
    "through","their","they","you","your","we","our"
])

def tokenize(text: str):
    return [
        t for t in re.findall(r"[A-Za-z][A-Za-z0-9\-_\’']+", text.lower())
        if t not in STOPWORDS
    ]


#####################################################################
# 6. FINAL EVALUATION FUNCTION  (FULL AND CORRECT)
#####################################################################
def evaluate(df_out: pd.DataFrame, df_ref: pd.DataFrame):

    rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    smooth = SmoothingFunction().method1

    sum_r, sum_b, sum_bert = [], [], []
    overlap_acc_list, jaccard_list, micro_f1_list = [], [], []
    macro_f1_list, weighted_f1_list = [], []
    qa_bleu, qa_div, qa_ans = [], [], []
    kc_p, kc_r, kc_f = [], [], []

    VALID_TOPICS = [
        "Natural Language Processing", "Artificial Intelligence", "Prompt Engineering",
        "Machine Learning", "Deep Learning", "Reinforcement Learning", "Generative AI",
        "Data Science", "Time Series", "Statistics", "LangChain", "Langraph",
        "Python Programming", "Mlops", "Agentic AI", "Other"
    ]

    # for macro/weighted F1
    all_true, all_pred = [], []

    for _, row in df_out.iterrows():
        idx = int(row["row_index"])
        ref_summary = df_ref.loc[idx, "Reference Summary"] or ""

        # -------------------- Summarisation --------------------
        gen_sum = row["summary"] or ""
        r = rouge.score(ref_summary, gen_sum)['rougeL'].fmeasure
        b = sentence_bleu([ref_summary.split()], gen_sum.split(), smoothing_function=smooth)

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            P, R, F1 = bert_score([gen_sum], [ref_summary], lang='en', verbose=False)

        sum_r.append(r)
        sum_b.append(b)
        sum_bert.append(float(F1.mean()))

        # -------------------- Topic Classification --------------------
        gold = gold_topics_from_ref_summary(ref_summary)
        pred = [x.strip() for x in (row["topic_classification"] or "").split(",") if x.strip()]

        set_pred = set(pred)
        set_gold = set(gold)

        # Overlap Accuracy (your metric)
        overlap_acc = 1.0 if len(set_pred & set_gold) > 0 else 0.0

        # Jaccard
        inter = len(set_pred & set_gold)
        union = len(set_pred | set_gold)
        jaccard = inter / union if union > 0 else 0.0

        # Micro-F1
        tp = inter
        fp = len([p for p in pred if p not in gold])
        fn = len([g for g in gold if g not in pred])

        prec = tp / (tp + fp) if (tp + fp) else 0.0
        rec  = tp / (tp + fn) if (tp + fn) else 0.0
        micro_f1 = (2 * prec * rec / (prec + rec)) if (prec + rec) else 0.0

        overlap_acc_list.append(overlap_acc)
        jaccard_list.append(jaccard)
        micro_f1_list.append(micro_f1)

        # Macro/Weighted F1 prep
        true_bin = [1 if t in gold else 0 for t in VALID_TOPICS]
        pred_bin = [1 if t in pred else 0 for t in VALID_TOPICS]

        all_true.append(true_bin)
        all_pred.append(pred_bin)

        # -------------------- Q&A --------------------
        qa_text = row["Q_and_A"] or ""
        qs = [l[2:].strip() for l in qa_text.splitlines() if l.lower().startswith("q:")]

        gold_qs = [
            "What is the main topic discussed in the video?",
            "Why is this topic important?",
            "How is the core concept explained?",
            "What example is mentioned in the content?",
            "What is the key conclusion of the video?"
        ]

        if qs:
            bleu_vals = [
                sentence_bleu([g.split()], q.split(), smoothing_function=smooth)
                for g in gold_qs for q in qs
            ]
            qa_bleu.append(np.mean(bleu_vals))
        else:
            qa_bleu.append(0.0)

        toks = [t for q in qs for t in q.split()]
        qa_div.append(len(set(toks)) / len(toks) if toks else 0.0)

        ref_tokens = set(tokenize(ref_summary))
        ans_count = sum(
            1 for q in qs
            if len(set(tokenize(q)) & ref_tokens) / max(1, len(tokenize(q))) >= 0.3
        )
        qa_ans.append(ans_count / len(qs) if qs else 0.0)

        # -------------------- Key Concepts --------------------
        kc_text = str(row.get("key_concepts", "") or "")
        pred_concepts = [c.strip().lower() for c in kc_text.split(",") if c.strip()]

        ref_concepts = tokenize(ref_summary)
        ref_top = ref_concepts[:25]

        tp_kc = len([p for p in pred_concepts[:10] if any(p in r or r in p for r in ref_top)])

        p_val = tp_kc / 10
        r_val = tp_kc / len(ref_top) if ref_top else 0
        f1_val = (2*p_val*r_val/(p_val+r_val)) if (p_val+r_val) else 0

        kc_p.append(p_val)
        kc_r.append(r_val)
        kc_f.append(f1_val)

    # Compute macro/weighted F1
    all_true = np.array(all_true)
    all_pred = np.array(all_pred)

    macro_f1 = precision_recall_fscore_support(all_true, all_pred, average="macro", zero_division=0)[2]
    weighted_f1 = precision_recall_fscore_support(all_true, all_pred, average="weighted", zero_division=0)[2]

    return {
        "Summarisation": {
            "ROUGE-L F1": float(np.mean(sum_r)),
            "BLEU": float(np.mean(sum_b)),
            "BERTScore F1": float(np.mean(sum_bert))
        },
        "Topic Classification": {
            "Overlap Accuracy": float(np.mean(overlap_acc_list)),
            "Jaccard Index": float(np.mean(jaccard_list)),
            "Micro F1": float(np.mean(micro_f1_list)),
            "Macro F1": float(macro_f1),
            "Weighted F1": float(weighted_f1)
        },
        "Q&A Generation": {
            "BLEU": float(np.mean(qa_bleu)),
            "Diversity": float(np.mean(qa_div)),
            "Answerability": float(np.mean(qa_ans))
        },
        "Key Concept Extraction": {
            "Precision@10": float(np.mean(kc_p)),
            "Recall@10": float(np.mean(kc_r)),
            "F1@10": float(np.mean(kc_f))
        }
    }


#####################################################################
# 7. RUN EVALUATION
#####################################################################
df_ref = pd.read_excel(INPUT_FILE)
df_out = pd.read_excel(OUTPUT_FILE)

eval_summary = evaluate(df_out, df_ref)

print("\n==================== FINAL EVALUATION METRICS ====================")
for task, vals in eval_summary.items():
    print(f"\n{task}:")
    for metric, value in vals.items():
        print(f"  - {metric}: {value:.4f}")

with open(FINAL_EVAL_JSON, "w") as f:
    json.dump(eval_summary, f, indent=2)

print("\nSaved corrected evaluation JSON to:", FINAL_EVAL_JSON)


Loaded input: /content/drive/MyDrive/Final Thesis Code/Input/clean_input_30.xlsx
Loaded model output: /content/drive/MyDrive/Final Thesis Code/Output/Zero Shot Prompting/llama-4-scout-17b-16e-instruct/llama-4-scout-17b-16e-instruct_zero_shot_full_output.xlsx


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]


==================== FINAL EVALUATION METRICS ====================

Summarisation:
  - ROUGE-L F1: 0.2397
  - BLEU: 0.0214
  - BERTScore F1: 0.8811

Topic Classification:
  - Overlap Accuracy: 0.9667
  - Jaccard Index: 0.4253
  - Micro F1: 0.5576
  - Macro F1: 0.5299
  - Weighted F1: 0.4910

Q&A Generation:
  - BLEU: 0.0476
  - Diversity: 0.7413
  - Answerability: 0.8333

Key Concept Extraction:
  - Precision@10: 0.5033
  - Recall@10: 0.2013
  - F1@10: 0.2876

Saved corrected evaluation JSON to: /content/drive/MyDrive/Final Thesis Code/Output/Zero Shot Prompting/llama-4-scout-17b-16e-instruct/evaluation_final.json
